# 1.生成文件


## 1.1 改造源程序

``` cpp

// 生成 treeADCnew.root 文件
void detector()
{
  const Double_t D=500.;//cm, distance between target and the scin.(Center)
  const Double_t L=100.;//cm, half length of the scin.
  const Double_t dD=5.;//cm, thickness of the scin.
  const Double_t TRes=1.;//ns, time resolution(FWHM) of the scintillator.
  const Double_t Lambda=380.;//cm, attenuation lenght of the scin.
  const Double_t QRes=0.1;//relative energy resolution(FWHM) of the scin. 
  const Double_t Vsc=7.5;//ns/cm, speed of light in the scin.
 //neutron & gamma
  const Double_t En0=100;//MeV, average neutron energy
  const Double_t EnRes=50.;//MeV, energy spread of neutron(FWHM)
  const Double_t Eg0=10;//MeV, gamma energy  

  const Double_t Rn=0.75;//ratio of neutron 
  const Double_t Rg=0.05;//ratio of gamma; 

 //charged particle hited on Neutron Detector, 
 const Double_t Rc=0.1;//ratio of proton
 const Double_t Ec0=50;//MeV
 const Double_t EcRes=50;//MeV energy spread of proton(FWHM)

 //ADC
 const Double_t ADCgain=60;//1MeV=60ch.
 const Double_t ADCuPed=100;//baseline of ADC for upper side
 const Double_t ADCdPed=120;//                for bottom side
 const Double_t ADCnoise=10;//sigma of noise
 const Int_t    ADCoverflow=4095;

 //TDC
 const Double_t TDCch2ns=40.;//1ns=40ch.
 const Int_t TDCoverflow=4095;
  
  TFile *opf=new TFile("treeADCnew.root","recreate");//新文件tree.root的指针 *opf
  TTree *opt=new TTree("tree","tree structure");//新tree的指针 *opt
    
  // 定义tree的branch变量
  Double_t x;
  Double_t e;
  int pid;    //0/1/2/3: gamma/neutron/proton/No_particle
  Double_t tof, ctof;
  Double_t tu, td;
  Double_t qu, qd;
  Int_t itu,itd;//TDC
  Int_t iqu,iqd;//ADC

  Double_t tup, tdp;
  Double_t qup, qdp;
  Int_t itup,itdp;//TDC
  Int_t iqup,iqdp;//ADC

  Double_t tu_off=5.5;//time offset
  Double_t td_off=20.4;//time offset
 
  // 将变量分支添加到tree结构中,第一个参数为变量名称，第二个为上面定义的变量地址，第三个为变量的类型说明，D表示Double_t。
  opt->Branch("x", &x, "x/D");//position
  opt->Branch("e", &e, "e/D");//energy
  opt->Branch("tof", &tof, "tof/D");//time of flight
  opt->Branch("ctof",&ctof,"ctof/D");//TOF from exp. data
  opt->Branch("pid", &pid, "pid/I");
  // raw time and energy
  opt->Branch("tu", &tu, "tu/D");
  opt->Branch("td", &td, "td/D");
  opt->Branch("qu", &qu, "qu/D");
  opt->Branch("qd", &qd, "qd/D");
  
  opt->Branch("tup", &tup, "tup/D");
  opt->Branch("tdp", &tdp, "tdp/D");
  opt->Branch("qup", &qup, "qup/D");
  opt->Branch("qdp", &qdp, "qdp/D"); 
    
    
  // energy in ADC, time in TDC 注意，以下Branch变量声明的类型为 Integer，
  opt->Branch("itu", &itu, "itu/I");
  opt->Branch("itd", &itd, "itd/I");
  opt->Branch("iqu", &iqu, "iqu/I");
  opt->Branch("iqd", &iqd, "iqd/I");

  opt->Branch("itup", &itup, "itup/I");
  opt->Branch("itdp", &itdp, "itdp/I");
  opt->Branch("iqup", &iqup, "iqup/I");
  opt->Branch("iqdp", &iqdp, "iqdp/I");
  
  TRandom3 *gr=new TRandom3(0);
  // 循环，逐事件往tree结构里添加对应分支信息。
  for(int i=0;i<1000000;i++)
  {
    x=gr->Uniform(-L, L);
    Double_t Dr=D+gr->Uniform(-0.5,0.5)*dD;
    Double_t d=TMath::Sqrt(Dr*Dr+x*x);//cm, flight path
    //pid
    Double_t ratio=gr->Uniform();
    if(ratio<Rg) pid=0;
    if(ratio>=Rg && ratio<Rg+Rn) pid=1;
    if(ratio>=Rg+Rn && ratio<Rg+Rn+Rc) pid=2;
    if(ratio>=Rg+Rn+Rc) pid=3;
    
    // energy & tof    
    if(pid==0) 
    {//gamma
       e=Eg0;
       tof=10/3.*(d*0.01);
      }
    if(pid==1) 
    { //neutron
       e=gr->Gaus(En0, EnRes/2.35); // neutron
       tof=72./TMath::Sqrt(e)*(d*0.01);//ns
    }
    if(pid==2) 
    {//proton
      e=gr->Gaus(Ec0, EcRes/2.35); // proton
      tof=72./TMath::Sqrt(e)*(d*0.01);
    }
    if(pid==3) 
    {
      e=-1;
      tof=-1;
    }
    
    //tu,td,qu,qd
    if(pid==3) 
    {//
      tu=TDCoverflow;
      td=TDCoverflow;
      qu=ADCuPed+gr->Gaus(0,ADCnoise);
      qd=ADCdPed+gr->Gaus(0,ADCnoise);
      
      tup=TDCoverflow;
      tdp=TDCoverflow; 
    }
    else 
    {
      //time
      tu=tof+(L-x)/Vsc+gr->Gaus(0,TRes/2.35)+tu_off;
      td=tof+(L+x)/Vsc+gr->Gaus(0,TRes/2.35)+td_off;
      tu *=TDCch2ns;
      td *=TDCch2ns;
      tup = tu;
      tdp = td;
      
      //energy depositon in the plastic
      Double_t q0, q0p;
      if(pid!=2) //neutron &gamma,
      {
         q0=e*ADCgain*gr->Uniform(); 
         tup = TDCoverflow;
         tdp = TDCoverflow;
      }
      else //charged particle
      {
          q0=0.9*e*ADCgain;
          q0p=0.1*e*ADCgain;
      }
      
      //resolution
      q0=gr->Gaus(q0,q0*QRes/2.35);
      q0p=gr->Gaus(q0p,q0p*QRes/2.35);
      
      //light transmission within the plastic
      qu=q0*TMath::Exp(-(L-x)/Lambda);
      qd=q0*TMath::Exp(-(L+x)/Lambda);
      qup=q0p*TMath::Exp(-(L-x)/Lambda);
      qdp=q0p*TMath::Exp(-(L+x)/Lambda);
      
      if(pid!=2)
      {
          qup = 0;
          qdp = 0;
      }
      
      //threshold of time of tu and td
      if(qu<100) tu=TDCoverflow;
      if(qd<100) td=TDCoverflow;
      if(qup<100) tup=TDCoverflow;
      if(qdp<100) tdp=TDCoverflow;

      //ADC
      qu += gr->Gaus(ADCuPed,ADCnoise);
      qd += gr->Gaus(ADCdPed,ADCnoise);
      qup += gr->Gaus(ADCuPed,ADCnoise);
      qdp += gr->Gaus(ADCdPed,ADCnoise);
      if(qu<0) qu=0;//no negative values
      if(qd<0) qd=0;
      if(qup<0) qup=0;//no negative values
      if(qdp<0) qdp=0;  
    }

      //overflow check
    if(tu>TDCoverflow) tu=TDCoverflow;
    if(td>TDCoverflow) td=TDCoverflow;
    if(qu>ADCoverflow) qu=ADCoverflow;
    if(qd>ADCoverflow) qd=ADCoverflow;
    if(tup>TDCoverflow) tup=TDCoverflow;
    if(tdp>TDCoverflow) tdp=TDCoverflow;
    if(qup>ADCoverflow) qup=ADCoverflow;
    if(qdp>ADCoverflow) qdp=ADCoverflow;
    
  //digitization
    itu=Int_t(tu);
    itd=Int_t(td);
    iqu=Int_t(qu);
    iqd=Int_t(qd);
    itup=Int_t(tup);
    itdp=Int_t(tdp);
    iqup=Int_t(qup);
    iqdp=Int_t(qdp);
    
    opt->Fill();
   
  }
  // 将数据写入root文件中
  opt->Write();
  opf->Close();
  
}

```

In [2]:
!touch detector.cpp 

In [3]:
!ls

'1.1 create tree.ipynb'		  coursework1.1.ipynb	 Readme.md
'1.2 read tree.ipynb'		  coursework1.2.ipynb	'ROOT tips.ipynb'
'1.3 adc overflow.ipynb'	  detector.cpp		 tof.png
'1.4 root-tree-makeclass.ipynb'   neutron_cali.pdf	 vetowall.png
 adc_ped.png			  neutron_response.pdf
 chapt1.2.ipynb			  nima-neutron.pdf



In [1]:
.L detector.cpp

In [2]:
detector()

In [1]:
%jsroot on

In [2]:
TFile *ipf=new TFile("treeADCnew.root");

In [3]:
tree->Print()

******************************************************************************
*Tree    :tree      : tree structure                                         *
*Entries :  1000000 : Total =       132160446 bytes  File  Size =   74774308 *
*        :          : Tree compression factor =   1.77                       *
******************************************************************************
*Br    0 :x         : x/D                                                    *
*Entries :  1000000 : Total  Size=    8009361 bytes  File Size  =    6129837 *
*Baskets :      103 : Basket Size=    3231744 bytes  Compression=   1.31     *
*............................................................................*
*Br    1 :e         : e/D                                                    *
*Entries :  1000000 : Total  Size=    8009361 bytes  File Size  =    6572222 *
*Baskets :      103 : Basket Size=    3231744 bytes  Compression=   1.22     *
*...................................................

In [4]:
TCanvas *c=new TCanvas();
c->Clear();

In [5]:
tree->Draw("iqup>>hqupall(120,0,1200)");
TH1D *hqupall=(TH1D*)gROOT->FindObject("hqupall");
hqupall->SetLineColor(kBlack);
tree->Draw("iqup>>hqup01(120,0,1200)","pid==0 || pid==1");
TH1D *hqup01=(TH1D*)gROOT->FindObject("hqup01");
hqup01->SetLineColor(kRed);
tree->Draw("iqup>>hqup2(120,0,1200)","pid==2");
TH1D *hqup2=(TH1D*)gROOT->FindObject("hqup2");
hqup2->SetLineColor(kGreen);
tree->Draw("iqup>>hqup3(120,0,1200)","pid==3");
TH1D *hqup3=(TH1D*)gROOT->FindObject("hqup3");
hqup3->SetLineColor(kBlue);
THStack *hsqup=new THStack("hsqup","qup with different pid");

In [6]:
hsqup->Add(hqupall);
hsqup->Add(hqup01);
hsqup->Add(hqup2);
hsqup->Add(hqup3);
hsqup->Draw("nostack");
gPad->SetLogy();
c->Draw();//辨认pedal峰，超界位置。

In [7]:
tree->Draw("itup>>htupall(270,1500,4200)");
TH1D *htupall=(TH1D*)gROOT->FindObject("htupall");
htupall->SetLineColor(kBlack);
tree->Draw("itup>>htup01(270,1500,4200)","pid==0 || pid==1");
TH1D *htup01=(TH1D*)gROOT->FindObject("htup01");
htup01->SetLineColor(kRed);
tree->Draw("itup>>htup2(270,1500,4200)","pid==2");
TH1D *htup2=(TH1D*)gROOT->FindObject("htup2");
htup2->SetLineColor(kGreen);
tree->Draw("itup>>htup3(270,1500,4200)","pid==3");
TH1D *htup3=(TH1D*)gROOT->FindObject("htup3");
htup3->SetLineColor(kBlue);
THStack *hstup=new THStack("hstup","tup with different pid");

In [8]:
hstup->Add(htupall);
hstup->Add(htup01);
hstup->Add(htup2);
hstup->Add(htup3);
hstup->Draw("nostack");
gPad->SetLogy();
c->Draw();//辨认pedal峰，超界位置。

## 1.1 使用 MakeClass 改变生成的root文件中的数据

In [1]:
%jsroot on

In [2]:
TFile *f=new TFile("treeADC.root");//root -l tree.root
f->ls()//ROOT 环境下 > .ls

TFile**		treeADC.root	
 TFile*		treeADC.root	
  KEY: TTree	tree;2	tree structure
  KEY: TTree	tree;1	tree structure


In [3]:
tree->Print()

******************************************************************************
*Tree    :tree      : tree structure                                         *
*Entries :  1000000 : Total =        84131176 bytes  File  Size =   56021256 *
*        :          : Tree compression factor =   1.50                       *
******************************************************************************
*Br    0 :x         : x/D                                                    *
*Entries :  1000000 : Total  Size=    8012177 bytes  File Size  =    6132298 *
*Baskets :      135 : Basket Size=    3961856 bytes  Compression=   1.31     *
*............................................................................*
*Br    1 :e         : e/D                                                    *
*Entries :  1000000 : Total  Size=    8012177 bytes  File Size  =    6577053 *
*Baskets :      135 : Basket Size=    3961856 bytes  Compression=   1.22     *
*...................................................

In [4]:
tree->MakeClass("ADCnew2");

Info in <TTreePlayer::MakeClass>: Files: ADCnew2.h and ADCnew2.C generated from TTree: tree


In [5]:
!ls

'1.1 create tree.ipynb'		  coursework1.1.ipynb	 tof.png
'1.2 read tree.ipynb'		  coursework1.2.ipynb	 treeADC.C
'1.3 adc overflow.ipynb'	  detector.cpp		 treeADCnew2.root
'1.4 root-tree-makeclass.ipynb'   jupyter.ipynb		 treeADCnew.root
 ADCnew2.C			  neutron_cali.pdf	 treeADC.root
 ADCnew2.h			  neutron_response.pdf	 tree.C
 adc_ped.png			  nima-neutron.pdf	 tree.root
 chapt1.1.ipynb			  Readme.md		 Untitled.ipynb
 chapt1.2.ipynb			 'ROOT tips.ipynb'	 vetowall.png



In [4]:
!cat ADCnew2.h

//////////////////////////////////////////////////////////
// This class has been automatically generated on
// Thu Mar 12 19:44:54 2020 by ROOT version 6.18/04
// from TTree tree/tree structure
// found on file: treeADC.root
//////////////////////////////////////////////////////////

#ifndef ADCnew2_h
#define ADCnew2_h

#include <TROOT.h>
#include <TChain.h>
#include <TFile.h>

// Header file for the classes stored in the TTree if any.

class ADCnew2 {
public :
   TTree          *fChain;   //!pointer to the analyzed TTree or TChain
   Int_t           fCurrent; //!current Tree number in a TChain

// Fixed size dimensions of array or collections stored in the TTree if any.

   // Declaration of leaf types
   Double_t        x;
   Double_t        e;
   Double_t        tof;
   Double_t        ctof;
   Int_t           pid;
   Double_t        tu;
   Double_t        td;
   Double_t        qu;
   Double_t        qd;
   Int_t           itu;
   UInt_t          itd;
   UInt_t          iqu;
   UI

In [5]:
!cat ADCnew2.C

#define ADCnew2_cxx
#include "ADCnew2.h"
#include <TH2.h>
#include <TStyle.h>
#include <TCanvas.h>

void ADCnew2::Loop()
{
//   In a ROOT session, you can do:
//      root> .L ADCnew2.C
//      root> ADCnew2 t
//      root> t.GetEntry(12); // Fill t data members with entry number 12
//      root> t.Show();       // Show values of entry 12
//      root> t.Show(16);     // Read and show values of entry 16
//      root> t.Loop();       // Loop on all entries
//

//     This is the loop skeleton where:
//    jentry is the global entry number in the chain
//    ientry is the entry number in the current Tree
//  Note that the argument to GetEntry must be:
//    jentry for TChain::GetEntry
//    ientry for TTree::GetEntry and TBranch::GetEntry
//
//       To read only selected branches, Insert statements like:
// METHOD1:
//    fChain->SetBranchStatus("*",0);  // disable all branches
//    fChain->SetBranchStatus("branchname",1);  // activate branchname
// METHOD2: replace line
//    fChain->

In [6]:
!ls

'1.1 create tree.ipynb'		  coursework1.1.ipynb	 tof.png
'1.2 read tree.ipynb'		  coursework1.2.ipynb	 treeADC.C
'1.3 adc overflow.ipynb'	  detector.cpp		 treeADCnew2.root
'1.4 root-tree-makeclass.ipynb'   jupyter.ipynb		 treeADCnew.root
 ADCnew2.C			  neutron_cali.pdf	 treeADC.root
 ADCnew2.h			  neutron_response.pdf	 tree.C
 adc_ped.png			  nima-neutron.pdf	 tree.root
 chapt1.1.ipynb			  Readme.md		 Untitled.ipynb
 chapt1.2.ipynb			 'ROOT tips.ipynb'	 vetowall.png



In [7]:
!cat ADCnew2.C

#define ADCnew2_cxx
#include "ADCnew2.h"
#include <TH2.h>
#include <TStyle.h>
#include <TCanvas.h>

void ADCnew2::Loop()
{
//   In a ROOT session, you can do:
//      root> .L ADCnew2.C
//      root> ADCnew2 t
//      root> t.GetEntry(12); // Fill t data members with entry number 12
//      root> t.Show();       // Show values of entry 12
//      root> t.Show(16);     // Read and show values of entry 16
//      root> t.Loop();       // Loop on all entries
//

//     This is the loop skeleton where:
//    jentry is the global entry number in the chain
//    ientry is the entry number in the current Tree
//  Note that the argument to GetEntry must be:
//    jentry for TChain::GetEntry
//    ientry for TTree::GetEntry and TBranch::GetEntry
//
//       To read only selected branches, Insert statements like:
// METHOD1:
//    fChain->SetBranchStatus("*",0);  // disable all branches
//    fChain->SetBranchStatus("branchname",1);  // activate branchname
// METHOD2: replace line
//    fChain->

In [10]:
.L ADCnew2.C

In [11]:
ADCnew2 t

(ADCnew2 &) @0x7f73fe670020


In [12]:
t.Loop();

In [13]:
!ls

'1.1 create tree.ipynb'		  coursework1.1.ipynb	 tof.png
'1.2 read tree.ipynb'		  coursework1.2.ipynb	 treeADC.C
'1.3 adc overflow.ipynb'	  detector.cpp		 treeADCnew2.root
'1.4 root-tree-makeclass.ipynb'   jupyter.ipynb		 treeADCnew.root
 ADCnew2.C			  neutron_cali.pdf	 treeADC.root
 ADCnew2.h			  neutron_response.pdf	 tree.C
 adc_ped.png			  nima-neutron.pdf	 tree.root
 chapt1.1.ipynb			  Readme.md		 Untitled.ipynb
 chapt1.2.ipynb			 'ROOT tips.ipynb'	 vetowall.png



In [1]:
%jsroot on

In [2]:
TFile *ipf1=new TFile("treeADCnew2.root");

In [3]:
tree->Print()

******************************************************************************
*Tree    :tree      : tree structure                                         *
*Entries :  1000000 : Total =       132157486 bytes  File  Size =   75964679 *
*        :          : Tree compression factor =   1.74                       *
******************************************************************************
*Br    0 :x         : x/D                                                    *
*Entries :  1000000 : Total  Size=    8009185 bytes  File Size  =    6129148 *
*Baskets :      101 : Basket Size=    3167744 bytes  Compression=   1.31     *
*............................................................................*
*Br    1 :e         : e/D                                                    *
*Entries :  1000000 : Total  Size=    8009185 bytes  File Size  =    6572501 *
*Baskets :      101 : Basket Size=    3167744 bytes  Compression=   1.22     *
*...................................................

In [4]:
TCanvas *c=new TCanvas();
c->Clear();

In [5]:
tree->Draw("iqup>>hqupall(120,0,1200)");
TH1D *hqupall=(TH1D*)gROOT->FindObject("hqupall");
hqupall->SetLineColor(kBlack);
tree->Draw("iqup>>hqup01(120,0,1200)","pid==0 || pid==1");
TH1D *hqup01=(TH1D*)gROOT->FindObject("hqup01");
hqup01->SetLineColor(kRed);
tree->Draw("iqup>>hqup2(120,0,1200)","pid==2");
TH1D *hqup2=(TH1D*)gROOT->FindObject("hqup2");
hqup2->SetLineColor(kGreen);
tree->Draw("iqup>>hqup3(120,0,1200)","pid==3");
TH1D *hqup3=(TH1D*)gROOT->FindObject("hqup3");
hqup3->SetLineColor(kBlue);
THStack *hsqup=new THStack("hsqup","qup with different pid");

In [8]:
hsqup->Add(hqupall);
hsqup->Add(hqup01);
hsqup->Add(hqup2);
hsqup->Add(hqup3);
hsqup->Draw("nostack");
gPad->SetLogy();
c->Draw();//辨认pedal峰，超界位置。

In [9]:
tree->Draw("itup>>htupall(270,1500,4200)");
TH1D *htupall=(TH1D*)gROOT->FindObject("htupall");
htupall->SetLineColor(kBlack);
tree->Draw("itup>>htup01(270,1500,4200)","pid==0 || pid==1");
TH1D *htup01=(TH1D*)gROOT->FindObject("htup01");
htup01->SetLineColor(kRed);
tree->Draw("itup>>htup2(270,1500,4200)","pid==2");
TH1D *htup2=(TH1D*)gROOT->FindObject("htup2");
htup2->SetLineColor(kGreen);
tree->Draw("itup>>htup3(270,1500,4200)","pid==3");
TH1D *htup3=(TH1D*)gROOT->FindObject("htup3");
htup3->SetLineColor(kBlue);
THStack *hstup=new THStack("hstup","tup with different pid");

In [10]:
hstup->Add(htupall);
hstup->Add(htup01);
hstup->Add(htup2);
hstup->Add(htup3);
hstup->Draw("nostack");
gPad->SetLogy();
c->Draw();//辨认pedal峰，超界位置。

# 1.验证薄塑料闪烁体可作为带电粒子的veto探测器

In [1]:
%jsroot on

In [2]:
TFile *ipf=new TFile("treeADCnew2.root");

In [3]:
tree->Print()

******************************************************************************
*Tree    :tree      : tree structure                                         *
*Entries :  1000000 : Total =       132157486 bytes  File  Size =   75964679 *
*        :          : Tree compression factor =   1.74                       *
******************************************************************************
*Br    0 :x         : x/D                                                    *
*Entries :  1000000 : Total  Size=    8009185 bytes  File Size  =    6129148 *
*Baskets :      101 : Basket Size=    3167744 bytes  Compression=   1.31     *
*............................................................................*
*Br    1 :e         : e/D                                                    *
*Entries :  1000000 : Total  Size=    8009185 bytes  File Size  =    6572501 *
*Baskets :      101 : Basket Size=    3167744 bytes  Compression=   1.22     *
*...................................................

In [4]:
TCanvas *c=new TCanvas();
c->Clear();

In [5]:
tree->Draw("iqu>>hquall(420,0,4200)");
TH1D *hquall=(TH1D*)gROOT->FindObject("hquall");
hquall->SetLineColor(kBlack);
tree->Draw("iqu>>hqu01(420,0,4200)","pid==0 || pid==1");
TH1D *hqu01=(TH1D*)gROOT->FindObject("hqu01");
hqu01->SetLineColor(kRed);
tree->Draw("iqu>>hqu2(420,0,4200)","pid==2");
TH1D *hqu2=(TH1D*)gROOT->FindObject("hqu2");
hqu2->SetLineColor(kGreen);
tree->Draw("iqu>>hqu3(420,0,4200)","pid==3");
TH1D *hqu3=(TH1D*)gROOT->FindObject("hqu3");
hqu3->SetLineColor(kBlue);
THStack *hsqu=new THStack("hsqu","qu with different pid");

In [10]:
hsqu->Add(hquall);
hsqu->Add(hqu01);
hsqu->Add(hqu2);
hsqu->Add(hqu3);
hsqu->Draw("nostack");
gPad->SetLogy();
c->Draw();//辨认pedal峰，超界位置。

In [11]:
hquall->Fit("gaus","","",0,250);
c->Draw();

 FCN=35373.4 FROM MIGRAD    STATUS=CONVERGED     268 CALLS         269 TOTAL
                     EDM=1.19935e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     3.83617e+04   1.49917e+02   1.06786e+01   2.63342e-06
   2  Mean         1.01213e+02   3.62143e-02   3.15023e-03   8.58656e-03
   3  Sigma        1.13147e+01   2.78775e-02   2.77134e-05  -2.34290e-01


In [12]:
tree->Draw("itu>>htuall(420,0,4200)");
TH1D *htuall=(TH1D*)gROOT->FindObject("htuall");
htuall->SetLineColor(kBlack);
tree->Draw("itu>>htu01(420,0,4200)","pid==0 || pid==1");
TH1D *htu01=(TH1D*)gROOT->FindObject("htu01");
htu01->SetLineColor(kRed);
tree->Draw("itu>>htu2(420,0,4200)","pid==2");
TH1D *htu2=(TH1D*)gROOT->FindObject("htu2");
htu2->SetLineColor(kGreen);
tree->Draw("itu>>htu3(420,0,4200)","pid==3");
TH1D *htu3=(TH1D*)gROOT->FindObject("htu3");
htu3->SetLineColor(kBlue);
THStack *hstu=new THStack("hstu","tu with different pid");

In [13]:
hstu->Add(htuall);
hstu->Add(htu01);
hstu->Add(htu2);
hstu->Add(htu3);
hstu->Draw("nostack");
gPad->SetLogy();
c->Draw();//辨认pedal峰，超界位置。

In [14]:
tree->Draw("iqd>>hqdall(420,0,4200)");
TH1D *hqdall=(TH1D*)gROOT->FindObject("hqdall");
hqdall->SetLineColor(kBlack);
tree->Draw("iqd>>hqd01(420,0,4200)","pid==0 || pid==1");
TH1D *hqd01=(TH1D*)gROOT->FindObject("hqd01");
hqd01->SetLineColor(kRed);
tree->Draw("iqd>>hqd2(420,0,4200)","pid==2");
TH1D *hqd2=(TH1D*)gROOT->FindObject("hqd2");
hqd2->SetLineColor(kGreen);
tree->Draw("iqd>>hqd3(420,0,4200)","pid==3");
TH1D *hqd3=(TH1D*)gROOT->FindObject("hqd3");
hqd3->SetLineColor(kBlue);
THStack *hsqd=new THStack("hsqd","qd with different pid");

In [15]:
hsqd->Add(hqdall);
hsqd->Add(hqd01);
hsqd->Add(hqd2);
hsqd->Add(hqd3);
hsqd->Draw("nostack");
gPad->SetLogy();
c->Draw();//辨认pedal峰，超界位置。

In [16]:
hqdall->Fit("gaus","","",50,150);
c->Draw();

 FCN=592.848 FROM MIGRAD    STATUS=CONVERGED      53 CALLS          54 TOTAL
                     EDM=4.28287e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     3.89156e+04   1.48738e+02   1.40853e+00   6.84551e-06
   2  Mean         1.21044e+02   3.53881e-02   4.08668e-04   1.26213e-02
   3  Sigma        1.11148e+01   2.66109e-02   7.41277e-06   5.57836e-01


In [17]:
tree->Draw("itd>>htdall(270,1500,4200)");
TH1D *htdall=(TH1D*)gROOT->FindObject("htdall");
htuall->SetLineColor(kBlack);
tree->Draw("itd>>htd01(270,1500,4200)","pid==0 || pid==1");
TH1D *htd01=(TH1D*)gROOT->FindObject("htd01");
htd01->SetLineColor(kRed);
tree->Draw("itd>>htd2(270,1500,4200)","pid==2");
TH1D *htd2=(TH1D*)gROOT->FindObject("htd2");
htd2->SetLineColor(kGreen);
tree->Draw("itd>>htd3(270,1500,4200)","pid==3");
TH1D *htd3=(TH1D*)gROOT->FindObject("htd3");
htd3->SetLineColor(kBlue);
THStack *hstd=new THStack("hstd","td with different pid");

In [18]:
hstd->Add(htdall);
hstd->Add(htd01);
hstd->Add(htd2);
hstd->Add(htd3);
hstd->Draw("nostack");
gPad->SetLogy();
c->Draw();

In [19]:
TF1 *fgaus[4];
Double_t ped[4],sigma[4];//u,d
TString sq[4]={"qu","qd","qup","qdp"};

In [20]:
fgaus[0]=hquall->GetFunction("gaus");
fgaus[1]=hqdall->GetFunction("gaus");
for(int i=0;i<2;i++) 
{
    ped[i]=fgaus[i]->GetParameter(1);
    sigma[i]=fgaus[i]->GetParameter(2);
    //TString的格式化输出。用法与printf一致。复杂格式输出推荐用TString::Form()。
    TString ss;
    ss.Form("ped_%s=%.2f, sigma_%s=%.2f",sq[i].Data(),ped[i],sq[i].Data(),sigma[i]); //设置输出格式
    cout<<ss<<endl;
}

ped_qu=101.21, sigma_qu=11.31
ped_qd=121.04, sigma_qd=11.11


In [21]:
tree->Draw("iqup>>hqupall(120,0,1200)");
TH1D *hqupall=(TH1D*)gROOT->FindObject("hqupall");
hqupall->SetLineColor(kBlack);
tree->Draw("iqup>>hqup01(120,0,1200)","pid==0 || pid==1");
TH1D *hqup01=(TH1D*)gROOT->FindObject("hqup01");
hqup01->SetLineColor(kRed);
tree->Draw("iqup>>hqup2(120,0,1200)","pid==2");
TH1D *hqup2=(TH1D*)gROOT->FindObject("hqup2");
hqup2->SetLineColor(kGreen);
tree->Draw("iqup>>hqup3(120,0,1200)","pid==3");
TH1D *hqup3=(TH1D*)gROOT->FindObject("hqup3");
hqup3->SetLineColor(kBlue);
THStack *hsqup=new THStack("hsqup","qup with different pid");

In [22]:
hsqup->Add(hqupall);
hsqup->Add(hqup01);
hsqup->Add(hqup2);
hsqup->Add(hqup3);
hsqup->Draw("nostack");
gPad->SetLogy();
c->Draw();//辨认pedal峰，超界位置。

  - 由上图可得：塑料闪烁体上端收集的电荷量可以大致区分带点粒子（P）和其他粒子，其他粒子穿过闪烁体探测器时，闪烁体探测器上端的电荷量为 pedal + 噪声，带电粒子穿过闪烁体探测器时，闪烁体探测器上端有信号，且信号搞于噪声。

In [23]:
hqupall->Fit("gaus","","",0,250);
c->Draw();

 FCN=19702.4 FROM MIGRAD    STATUS=CONVERGED     117 CALLS         118 TOTAL
                     EDM=9.95031e-08    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   2.1 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     3.43458e+05   4.45902e+02  -1.46737e+01   8.33215e-07
   2  Mean         1.00015e+02   1.10116e-02   1.23429e-04  -7.15626e-03
   3  Sigma        1.04818e+01   8.01492e-03   4.64318e-06   3.94012e+00


In [24]:
tree->Draw("itup>>htupall(270,1500,4200)");
TH1D *htupall=(TH1D*)gROOT->FindObject("htupall");
htupall->SetLineColor(kBlack);
tree->Draw("itup>>htup01(270,1500,4200)","pid==0 || pid==1");
TH1D *htup01=(TH1D*)gROOT->FindObject("htup01");
htup01->SetLineColor(kRed);
tree->Draw("itup>>htup2(270,1500,4200)","pid==2");
TH1D *htup2=(TH1D*)gROOT->FindObject("htup2");
htup2->SetLineColor(kGreen);
tree->Draw("itup>>htup3(270,1500,4200)","pid==3");
TH1D *htup3=(TH1D*)gROOT->FindObject("htup3");
htup3->SetLineColor(kBlue);
THStack *hstup=new THStack("hstup","tup with different pid");

In [25]:
hstup->Add(htupall);
hstup->Add(htup01);
hstup->Add(htup2);
hstup->Add(htup3);
hstup->Draw("nostack");
gPad->SetLogy();
c->Draw();//辨认pedal峰，超界位置。

    - 由上图可得：闪烁体探测器的上端的时间信号可以对带点粒子（p）和其他粒子进行区分，其他粒子的时间信号超界，均为4095，故上端时间不是4095，即为带电粒子（P）

In [26]:
tree->Draw("iqdp>>hqdpall(120,0,1200)");
TH1D *hqdpall=(TH1D*)gROOT->FindObject("hqdpall");
hqdpall->SetLineColor(kBlack);
tree->Draw("iqdp>>hqdp01(120,0,1200)","pid==0 || pid==1");
TH1D *hqdp01=(TH1D*)gROOT->FindObject("hqdp01");
hqdp01->SetLineColor(kRed);
tree->Draw("iqdp>>hqdp2(120,0,1200)","pid==2");
TH1D *hqdp2=(TH1D*)gROOT->FindObject("hqdp2");
hqdp2->SetLineColor(kGreen);
tree->Draw("iqdp>>hqdp3(120,0,1200)","pid==3");
TH1D *hqdp3=(TH1D*)gROOT->FindObject("hqdp3");
hqdp3->SetLineColor(kBlue);
THStack *hsqdp=new THStack("hsqdp","qdp with different pid");

In [27]:
hsqdp->Add(hqdpall);
hsqdp->Add(hqdp01);
hsqdp->Add(hqdp2);
hsqdp->Add(hqdp3);
hsqdp->Draw("nostack");
gPad->SetLogy();
c->Draw();//辨认pedal峰，超界位置。

  - 由上图可得：塑料闪烁体下端收集的电荷量可以大致区分带点粒子（P）和其他粒子，其他粒子穿过闪烁体探测器时，闪烁体探测器下端的电荷量为 pedal + 噪声，带电粒子穿过闪烁体探测器时，闪烁体探测器下端有信号，且信号搞于噪声。

In [28]:
hqdpall->Fit("gaus","","",0,250);
c->Draw();

 FCN=14132.2 FROM MIGRAD    STATUS=CONVERGED      97 CALLS          98 TOTAL
                     EDM=5.21001e-11    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   2.4 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     3.44020e+05   4.46996e+02  -3.10367e+00   2.82212e-08
   2  Mean         1.20024e+02   1.10166e-02  -1.72047e-06  -4.08167e-04
   3  Sigma        1.04640e+01   8.01082e-03   2.20560e-06   7.46997e-02


In [29]:
tree->Draw("itdp>>htdpall(270,1500,4200)");
TH1D *htdpall=(TH1D*)gROOT->FindObject("htdpall");
htdpall->SetLineColor(kBlack);
tree->Draw("itdp>>htdp01(270,1500,4200)","pid==0 || pid==1");
TH1D *htdp01=(TH1D*)gROOT->FindObject("htdp01");
htdp01->SetLineColor(kRed);
tree->Draw("itdp>>htdp2(270,1500,4200)","pid==2");
TH1D *htdp2=(TH1D*)gROOT->FindObject("htdp2");
htdp2->SetLineColor(kGreen);
tree->Draw("itdp>>htdp3(270,1500,4200)","pid==3");
TH1D *htdp3=(TH1D*)gROOT->FindObject("htdp3");
htdp3->SetLineColor(kBlue);
THStack *hstdp=new THStack("hstdp","tdp with different pid");

In [30]:
hstdp->Add(htdpall);
hstdp->Add(htdp01);
hstdp->Add(htdp2);
hstdp->Add(htdp3);
hstdp->Draw("nostack");
gPad->SetLogy();
c->Draw();//辨认pedal峰，超界位置。

   - 由上图可得：闪烁体探测器的下端的时间信号可以对带点粒子（p）和其他粒子进行区分，其他粒子的时间信号超界，均为4095，故下端时间不是4095，即为带电粒子（P）

In [31]:
fgaus[2]=hqupall->GetFunction("gaus");
fgaus[3]=hqdpall->GetFunction("gaus");
for(int i=2;i<4;i++) 
{
    ped[i]=fgaus[i]->GetParameter(1);
    sigma[i]=fgaus[i]->GetParameter(2);
    //TString的格式化输出。用法与printf一致。复杂格式输出推荐用TString::Form()。
    TString ss;
    ss.Form("ped_%s=%.2f, sigma_%s=%.2f",sq[i].Data(),ped[i],sq[i].Data(),sigma[i]); //设置输出格式
    cout<<ss<<endl;
}

ped_qup=100.01, sigma_qup=10.48
ped_qdp=120.02, sigma_qdp=10.46


In [32]:
TString stree,scut;

  - 利用闪烁体探测器上端的Q值，绘制带电粒子通过中子探测器时，其上端Q值柱状图

In [33]:
stree.Form("(iqu-%f)>>hsud(200,0,4000)",ped[0]);//在""内加入动态数据的方法  form设置string格式
scut.Form("iqup>(%f+3*%f) ",ped[2],sigma[2]);
tree->Draw(stree.Data(),scut.Data(),"colz");   //.Data为string标志符代表的数据 
c->Draw();
gPad->SetLogy();
cout<<"tree: "<<stree.Data()<<endl;
cout<<"cut: "<<scut.Data()<<endl;

tree: (iqu-101.212965)>>hsud(200,0,4000)
cut: iqup>(100.014826+3*10.481844) 


- 利用闪烁体探测器下端的Q值，绘制带电粒子通过中子探测器时，其上端Q值柱状图

In [34]:
stree.Form("(iqu-%f)>>hsud(200,0,4000)",ped[0]);//在""内加入动态数据的方法  form设置string格式
scut.Form("iqdp>(%f+3*%f) ",ped[3],sigma[3]);
tree->Draw(stree.Data(),scut.Data(),"colz");   //.Data为string标志符代表的数据 
c->Draw();
gPad->SetLogy();
cout<<"tree: "<<stree.Data()<<endl;
cout<<"cut: "<<scut.Data()<<endl;

tree: (iqu-101.212965)>>hsud(200,0,4000)
cut: iqdp>(120.024167+3*10.463976) 


- 利用闪烁体探测器上下端的Q值，绘制带电粒子通过中子探测器时，其上端Q值柱状图

In [35]:
stree.Form("(iqu-%f)>>hsud(200,0,4000)",ped[0]);//在""内加入动态数据的方法  form设置string格式
scut.Form("iqup>(%f+3*%f) && iqdp>(%f+3*%f) ",ped[2], sigma[2],ped[3],sigma[3]);
tree->Draw(stree.Data(),scut.Data(),"colz");   //.Data为string标志符代表的数据 
c->Draw();
gPad->SetLogy();
cout<<"tree: "<<stree.Data()<<endl;
cout<<"cut: "<<scut.Data()<<endl;

tree: (iqu-101.212965)>>hsud(200,0,4000)
cut: iqup>(100.014826+3*10.481844) && iqdp>(120.024167+3*10.463976) 


- 利用闪烁体探测器上下端的Q值，绘制带电粒子通过中子探测器时，其上下端Q值的关联图

In [36]:
stree.Form("(iqu-%f):(iqd-%f)>>hsud(200,0,4000,200,0,4000)",ped[0],ped[1]);//在""内加入动态数据的方法  form设置string格式
scut.Form("iqup>(%f+3*%f) && iqdp>(%f+3*%f) ",ped[2], sigma[2],ped[3],sigma[3]);
tree->Draw(stree.Data(),scut.Data(),"colz");   //.Data为string标志符代表的数据 
gPad->SetLogy(0);
c->Draw();
cout<<"tree: "<<stree.Data()<<endl;
cout<<"cut: "<<scut.Data()<<endl;

tree: (iqu-101.212965):(iqd-121.043671)>>hsud(200,0,4000,200,0,4000)
cut: iqup>(100.014826+3*10.481844) && iqdp>(120.024167+3*10.463976) 


  - 利用闪烁体探测器上端的t值，绘制带电粒子通过中子探测器时，其上端Q值柱状图

In [37]:
stree.Form("(iqu-%f)>>hsud(200,0,4000)",ped[0]);//在""内加入动态数据的方法  form设置string格式
scut.Form("itup != 4095");
tree->Draw(stree.Data(),scut.Data(),"colz");   //.Data为string标志符代表的数据 
c->Draw();
gPad->SetLogy();
cout<<"tree: "<<stree.Data()<<endl;
cout<<"cut: "<<scut.Data()<<endl;

tree: (iqu-101.212965)>>hsud(200,0,4000)
cut: itup != 4095


  - 利用闪烁体探测器下端的t值，绘制带电粒子通过中子探测器时，其上端Q值柱状图

In [38]:
stree.Form("(iqu-%f)>>hsud(200,0,4000)",ped[0]);//在""内加入动态数据的方法  form设置string格式
scut.Form("itdp != 4095");
tree->Draw(stree.Data(),scut.Data(),"colz");   //.Data为string标志符代表的数据 
c->Draw();
gPad->SetLogy();
cout<<"tree: "<<stree.Data()<<endl;
cout<<"cut: "<<scut.Data()<<endl;

tree: (iqu-101.212965)>>hsud(200,0,4000)
cut: itdp != 4095


  - 利用闪烁体探测器上下端的t值，绘制带电粒子通过中子探测器时，其上端Q值柱状图

In [39]:
stree.Form("(iqu-%f)>>hsud(200,0,4000)",ped[0]);//在""内加入动态数据的方法  form设置string格式
scut.Form("itup != 4095 && itdp != 4095");
tree->Draw(stree.Data(),scut.Data(),"colz");   //.Data为string标志符代表的数据 
c->Draw();
gPad->SetLogy();
cout<<"tree: "<<stree.Data()<<endl;
cout<<"cut: "<<scut.Data()<<endl;

tree: (iqu-101.212965)>>hsud(200,0,4000)
cut: itup != 4095 && itdp != 4095


- 利用闪烁体探测器上下端的t值，绘制带电粒子通过中子探测器时，其上下端Q值的关联图

In [40]:
stree.Form("(iqu-%f):(iqd-%f)>>hsud(200,0,4000,200,0,4000)",ped[0],ped[1]);//在""内加入动态数据的方法  form设置string格式
scut.Form("itup != 4095 && itdp != 4095");
tree->Draw(stree.Data(),scut.Data(),"colz");   //.Data为string标志符代表的数据 
gPad->SetLogy(0);
c->Draw();
cout<<"tree: "<<stree.Data()<<endl;
cout<<"cut: "<<scut.Data()<<endl;

tree: (iqu-101.212965):(iqd-121.043671)>>hsud(200,0,4000,200,0,4000)
cut: itup != 4095 && itdp != 4095
